In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

df= pd.read_csv('C://Users//User//Desktop//MSc Westminster//Dissertation//DataSets//Heart_Disease_Indicators.csv')

In [2]:
from sklearn.model_selection import train_test_split

target_size = 50000

num_class_0 = int(target_size * df['HeartDiseaseorAttack'].value_counts(normalize=True)[0])
num_class_1 = target_size - num_class_0

df_class_0 = df[df['HeartDiseaseorAttack'] == 0]
df_class_1 = df[df['HeartDiseaseorAttack'] == 1]

df_class_0_sampled = df_class_0.sample(n=num_class_0, random_state=15)
df_class_1_sampled = df_class_1.sample(n=num_class_1, random_state=15)

df_sampled = pd.concat([df_class_0_sampled, df_class_1_sampled])

df_sampled = df_sampled.sample(frac=1, random_state=15).reset_index(drop=True)
df= df_sampled
print(df['HeartDiseaseorAttack'].value_counts(normalize=True))

0    0.90582
1    0.09418
Name: HeartDiseaseorAttack, dtype: float64


In [3]:
df.drop_duplicates(inplace= True)
df.shape

(48050, 22)

In [4]:
df= df[df['Diabetes'] != 1].copy()
df.loc[df['Diabetes'] == 2, 'Diabetes'] = 1
print(df['Diabetes'].value_counts())

0    40175
1     6968
Name: Diabetes, dtype: int64


In [5]:
categorical_columns= ['HighBP', 'HighChol', 'CholCheck', 'Smoker', 'Stroke', 'Diabetes', 'PhysActivity',
                      'Fruits', 'Veggies', 'HvyAlcoholConsump', 'AnyHealthcare', 'NoDocbcCost', 'DiffWalk',
                      'Sex', 'Age', 'Education', 'Income'
                     ]
df[categorical_columns]= df[categorical_columns].astype(str)
df= pd.get_dummies(df, columns= categorical_columns, drop_first= True)

In [6]:
X= df.drop(['HeartDiseaseorAttack'], axis= 1)
y= df['HeartDiseaseorAttack']

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test= train_test_split(X, y, test_size= 0.2, random_state= 15, stratify= y)

continuous_columns = ['BMI', 'GenHlth', 'MentHlth', 'PhysHlth']

from sklearn.preprocessing import MinMaxScaler
mms= MinMaxScaler()
X_train = mms.fit_transform(X_train)
X_test = mms.transform(X_test)

In [8]:
from imblearn.combine import SMOTETomek

smt= SMOTETomek(random_state= 15)
X_sm_tl, y_sm_tl= smt.fit_resample(X_train, y_train)

In [9]:
from sklearn.feature_selection import SelectKBest, chi2

selector= SelectKBest(score_func= chi2, k= 10)
X_sm_tl_selected= selector.fit_transform(X_sm_tl, y_sm_tl)
X_sm_tl_test_selected= selector.transform(X_test)

In [10]:
feature_names= X.columns
selected_indices= selector.get_support(indices=True)
selected_features= feature_names[selected_indices]
print("Selected Features: ", selected_features)

Selected Features:  Index(['GenHlth', 'PhysHlth', 'HighBP_1', 'HighChol_1', 'Stroke_1',
       'Diabetes_1', 'DiffWalk_1', 'Age_13', 'Age_5', 'Income_8'],
      dtype='object')


--- LogisticRegression ---

In [11]:
from sklearn.metrics import confusion_matrix, classification_report, precision_score, roc_auc_score, accuracy_score
from sklearn.linear_model import LogisticRegression
lr= LogisticRegression(random_state= 15)
lr.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lr= lr.predict(X_sm_tl_test_selected)
y_pred_prob_lr= lr.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lr, labels= lr.classes_)
print(classification_report(y_test, y_pred_lr, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lr, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lr))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lr))

              precision    recall  f1-score   support

           0       0.97      0.76      0.85      8517
           1       0.25      0.76      0.38       912

    accuracy                           0.76      9429
   macro avg       0.61      0.76      0.62      9429
weighted avg       0.90      0.76      0.81      9429

Precision: 0.2532991202346041
Accuracy: 0.7605260366953017
AUC: 0.8289333355991835


--- DecisionTreeClassifier ---

In [12]:
from sklearn.tree import DecisionTreeClassifier
dt= DecisionTreeClassifier(random_state=15)
dt.fit(X_sm_tl_selected, y_sm_tl)
y_pred_dt= dt.predict(X_sm_tl_test_selected)
y_pred_prob_dt= dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_dt, labels= dt.classes_)
print(classification_report(y_test, y_pred_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_dt))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      8517
           1       0.38      0.14      0.21       912

    accuracy                           0.89      9429
   macro avg       0.65      0.56      0.58      9429
weighted avg       0.86      0.89      0.87      9429

Precision: 0.3804034582132565
Accuracy: 0.894474493583625
AUC: 0.7008777208225447


--- Tuned - DecisionTreeClassifier ---

In [13]:
from sklearn.model_selection import GridSearchCV
param_grid= {
             'max_depth': [None, 10, 20, 30, 40, 50],
             'min_samples_split': [2, 10, 20],
             'min_samples_leaf': [1, 5, 10],
             'max_features': [None, 'sqrt', 'log2'],
             'criterion': ['gini', 'entropy']
            }

gs_dt= GridSearchCV(estimator= dt, param_grid= param_grid, cv= 5, scoring= 'precision')
gs_dt.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_dt.best_params_)
print("Best Precision Score:", gs_dt.best_score_)

Best Parameters: {'criterion': 'entropy', 'max_depth': None, 'max_features': None, 'min_samples_leaf': 1, 'min_samples_split': 2}
Best Precision Score: 0.9643611935848503


In [14]:
tuned_dt= gs_dt.best_estimator_
y_pred_tuned_dt= tuned_dt.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_dt= tuned_dt.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_dt, labels= tuned_dt.classes_)
print(classification_report(y_test, y_pred_tuned_dt, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_dt, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_dt))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_dt))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94      8517
           1       0.37      0.14      0.21       912

    accuracy                           0.89      9429
   macro avg       0.64      0.56      0.58      9429
weighted avg       0.86      0.89      0.87      9429

Precision: 0.3732193732193732
Accuracy: 0.8938381588715665
AUC: 0.6994773996897845


--- KNeighborsClassifier ---

In [15]:
from sklearn.neighbors import KNeighborsClassifier
knn= KNeighborsClassifier()
knn.fit(X_sm_tl_selected, y_sm_tl)
y_pred_knn= knn.predict(X_sm_tl_test_selected)
y_pred_prob_knn= knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_knn, labels= knn.classes_)
print(classification_report(y_test, y_pred_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_knn))

              precision    recall  f1-score   support

           0       0.92      0.95      0.93      8517
           1       0.32      0.21      0.25       912

    accuracy                           0.88      9429
   macro avg       0.62      0.58      0.59      9429
weighted avg       0.86      0.88      0.87      9429

Precision: 0.3163934426229508
Accuracy: 0.8795206278502492
AUC: 0.7174959291942431


--- Tuned - KNeighborsClassifier ---

In [16]:
param_grid= {
             'n_neighbors': np.arange(1,40),
             'weights': ['uniform', 'distance'],
             'metric': ['euclidean', 'manhattan', 'minkowski']
            }

gs_knn= GridSearchCV(estimator= knn, param_grid= param_grid, cv=5, scoring= 'precision')
gs_knn.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_knn.best_params_)
print("Best Precision Score:", gs_knn.best_score_)

Best Parameters: {'metric': 'manhattan', 'n_neighbors': 2, 'weights': 'uniform'}
Best Precision Score: 0.9707907332703606


In [17]:
tuned_knn= gs_knn.best_estimator_
y_pred_tuned_knn= tuned_knn.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_knn= tuned_knn.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_knn, labels= tuned_knn.classes_)
print(classification_report(y_test, y_pred_tuned_knn, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_knn, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_knn))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_knn))

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      8517
           1       0.38      0.13      0.20       912

    accuracy                           0.90      9429
   macro avg       0.65      0.56      0.57      9429
weighted avg       0.86      0.90      0.87      9429

Precision: 0.37888198757763975
Accuracy: 0.8950047725103404
AUC: 0.634219177743584


--- GaussianNB ---

In [18]:
from sklearn.naive_bayes import GaussianNB
nb= GaussianNB()
nb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_nb= nb.predict(X_sm_tl_test_selected)
y_pred_prob_nb= nb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_nb, labels= nb.classes_)
print(classification_report(y_test, y_pred_nb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_nb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_nb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_nb))

              precision    recall  f1-score   support

           0       0.97      0.73      0.83      8517
           1       0.24      0.78      0.37       912

    accuracy                           0.74      9429
   macro avg       0.60      0.76      0.60      9429
weighted avg       0.90      0.74      0.79      9429

Precision: 0.2390358218948778
Accuracy: 0.7379361544172235
AUC: 0.8146457343311313


--- SVM ---

In [19]:
from sklearn.svm import SVC
svc= SVC(kernel= 'rbf',probability= True, gamma= 1, random_state=15)
svc.fit(X_sm_tl_selected, y_sm_tl)
y_pred_svc= svc.predict(X_sm_tl_test_selected)
y_pred_prob_svc= svc.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_svc, labels= svc.classes_)
print(classification_report(y_test, y_pred_svc, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_svc, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_svc))
print('AUC:', roc_auc_score(y_test, y_pred_prob_svc))

              precision    recall  f1-score   support

           0       0.95      0.78      0.86      8517
           1       0.24      0.65      0.35       912

    accuracy                           0.77      9429
   macro avg       0.60      0.72      0.61      9429
weighted avg       0.89      0.77      0.81      9429

Precision: 0.24226384364820847
Accuracy: 0.7690104995227489
AUC: 0.775532655020197


--- Random Forest ---

In [20]:
from sklearn.ensemble import RandomForestClassifier
rf= RandomForestClassifier(random_state=15)
rf.fit(X_sm_tl_selected, y_sm_tl)
y_pred_rf= rf.predict(X_sm_tl_test_selected)
y_pred_prob_rf= rf.predict_proba(X_sm_tl_test_selected)[:,1]

In [21]:
cm= confusion_matrix(y_test, y_pred_rf, labels= rf.classes_)
print(classification_report(y_test, y_pred_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_rf))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94      8517
           1       0.40      0.19      0.26       912

    accuracy                           0.89      9429
   macro avg       0.66      0.58      0.60      9429
weighted avg       0.87      0.89      0.88      9429

Precision: 0.39770114942528734
Accuracy: 0.8938381588715665
AUC: 0.7487069205242766


In [22]:
param_grid= {
             'n_estimators': [100, 200, 300],
             'max_depth': [None, 10, 20, 30],
             'min_samples_split': [2, 5, 10],
             'min_samples_leaf': [1, 2, 4],
            }

gs_tuned_rf= GridSearchCV(estimator= rf, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_tuned_rf.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_tuned_rf.best_params_)
print("Best Precision Score:", gs_tuned_rf.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Best Parameters: {'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 300}
Best Precision Score: 0.9603680484816086


In [23]:
tuned_rf= gs_tuned_rf.best_estimator_
y_pred_tuned_rf= tuned_rf.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_rf= tuned_rf.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_rf, labels= tuned_rf.classes_)
print(classification_report(y_test, y_pred_tuned_rf, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_rf, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_rf))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_rf))

              precision    recall  f1-score   support

           0       0.92      0.97      0.94      8517
           1       0.38      0.18      0.24       912

    accuracy                           0.89      9429
   macro avg       0.65      0.57      0.59      9429
weighted avg       0.86      0.89      0.87      9429

Precision: 0.3818615751789976
Accuracy: 0.8927776010181355
AUC: 0.7497424526591812


--- AdaBoost ---

In [24]:
from sklearn.ensemble import AdaBoostClassifier
ada= AdaBoostClassifier(random_state=15)
ada.fit(X_sm_tl_selected, y_sm_tl)
y_pred_ada= ada.predict(X_sm_tl_test_selected)
y_pred_prob_ada= ada.predict_proba(X_sm_tl_test_selected)[:,1]

In [25]:
cm= confusion_matrix(y_test, y_pred_ada, labels= ada.classes_)
print(classification_report(y_test, y_pred_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_ada))

              precision    recall  f1-score   support

           0       0.95      0.87      0.91      8517
           1       0.31      0.54      0.39       912

    accuracy                           0.84      9429
   macro avg       0.63      0.71      0.65      9429
weighted avg       0.89      0.84      0.86      9429

Precision: 0.31069182389937106
Accuracy: 0.839431540990561
AUC: 0.8231227174134703


In [26]:
param_grid= {'n_estimators': [50, 100, 200]}

gs_ada= GridSearchCV(estimator= ada, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_ada.fit(X_sm_tl_selected, y_sm_tl)
print("Best Parameters:", gs_ada.best_params_)
print("Best Precision Score:", gs_ada.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'n_estimators': 200}
Best Precision Score: 0.9169691684701672


In [27]:
tuned_ada= gs_ada.best_estimator_
y_pred_tuned_ada= tuned_ada.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_ada= tuned_ada.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_ada, labels= tuned_ada.classes_)
print(classification_report(y_test, y_pred_tuned_ada, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_ada, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_ada))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_ada))

              precision    recall  f1-score   support

           0       0.93      0.93      0.93      8517
           1       0.36      0.38      0.37       912

    accuracy                           0.87      9429
   macro avg       0.65      0.65      0.65      9429
weighted avg       0.88      0.87      0.88      9429

Precision: 0.35908141962421714
Accuracy: 0.8746420617244671
AUC: 0.8223771111028717


--- GradientBoosting ---

In [28]:
from sklearn.ensemble import GradientBoostingClassifier
grb= GradientBoostingClassifier(random_state=15)
grb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_grb= grb.predict(X_sm_tl_test_selected)
y_pred_prob_grb= grb.predict_proba(X_sm_tl_test_selected)[:,1]


cm= confusion_matrix(y_test, y_pred_grb, labels= grb.classes_)
print(classification_report(y_test, y_pred_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_grb))

              precision    recall  f1-score   support

           0       0.93      0.92      0.93      8517
           1       0.34      0.37      0.36       912

    accuracy                           0.87      9429
   macro avg       0.64      0.65      0.64      9429
weighted avg       0.88      0.87      0.87      9429

Precision: 0.3413654618473896
Accuracy: 0.8697634955986849
AUC: 0.8169875419439759


In [29]:
param_grid= {'learning_rate': [0.01, 0.1, 0.2]}

gs_grb= GridSearchCV(estimator= grb, param_grid= param_grid, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_grb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_grb.best_params_)
print("Best Precision Score:", gs_grb.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 0.2}
Best Precision Score: 0.9535460446356219


In [30]:
tuned_grb= gs_grb.best_estimator_
y_pred_tuned_grb= tuned_grb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_grb= tuned_grb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_grb, labels= tuned_grb.classes_)
print(classification_report(y_test, y_pred_tuned_grb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_grb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_grb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_grb))

              precision    recall  f1-score   support

           0       0.92      0.96      0.94      8517
           1       0.40      0.26      0.32       912

    accuracy                           0.89      9429
   macro avg       0.66      0.61      0.63      9429
weighted avg       0.87      0.89      0.88      9429

Precision: 0.3963815789473684
Accuracy: 0.8899140948138721
AUC: 0.8102075647466676


--- XGB ---

In [31]:
from xgboost import XGBClassifier
xgb= XGBClassifier(random_state=15)
xgb.fit(X_sm_tl_selected, y_sm_tl)
y_pred_xgb= xgb.predict(X_sm_tl_test_selected)
y_pred_prob_xgb= xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_xgb, labels= xgb.classes_)
print(classification_report(y_test, y_pred_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_xgb))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.45      0.15      0.22       912

    accuracy                           0.90      9429
   macro avg       0.68      0.56      0.59      9429
weighted avg       0.87      0.90      0.88      9429

Precision: 0.45484949832775917
Accuracy: 0.900413617562838
AUC: 0.7933903864098428


In [32]:
params_XGBoost= {'learning_rate': [0.01, 0.1, 1.0]}

gs_xgb= GridSearchCV(estimator= xgb, param_grid= params_XGBoost, cv= 5, scoring= 'precision', n_jobs= -1, verbose= 2)
gs_xgb.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_xgb.best_params_)
print("Best Precision Score:", gs_xgb.best_score_)

tuned_xgb= gs_xgb.best_estimator_
y_pred_tuned_xgb= tuned_xgb.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_xgb= tuned_xgb.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_xgb, labels= tuned_xgb.classes_)
print(classification_report(y_test, y_pred_tuned_xgb, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_xgb, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_xgb))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_xgb))

Fitting 5 folds for each of 3 candidates, totalling 15 fits
Best Parameters: {'learning_rate': 1.0}
Best Precision Score: 0.9687779967110032
              precision    recall  f1-score   support

           0       0.92      0.97      0.94      8517
           1       0.38      0.16      0.22       912

    accuracy                           0.89      9429
   macro avg       0.65      0.56      0.58      9429
weighted avg       0.86      0.89      0.87      9429

Precision: 0.38235294117647056
Accuracy: 0.8939442146569095
AUC: 0.7699314992306409


--- LGBM ---

In [33]:
from lightgbm import LGBMClassifier
lgm= LGBMClassifier(random_state=15)
lgm.fit(X_sm_tl_selected, y_sm_tl)
y_pred_lgm= lgm.predict(X_sm_tl_test_selected)
y_pred_prob_lgm= lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_lgm, labels= lgm.classes_)
print(classification_report(y_test, y_pred_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_lgm))

[LightGBM] [Info] Number of positive: 33930, number of negative: 33930
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.005435 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 67860, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
              precision    recall  f1-score   support

           0       0.92      0.98      0.95      8517
           1       0.43      0.15      0.23       912

    accuracy                           0.90      9429
   macro avg       0.67      0.57      0.59      9429
weighted avg       0.87      0.90      0.88      9429

Precision: 0.4325153374233129
Accuracy: 0.8986106692120055
AUC: 0.801995982235735


In [34]:
params_LGB= {'learning_rate': [0.001, 0.01, 0.1, 1.0],
             'num_leaves': [31, 127],
             'reg_alpha': [0.1, 0.5],
             'min_data_in_leaf': [30, 50, 100, 300, 400]}

gs_lgm= GridSearchCV(estimator= lgm, param_grid= params_LGB, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_lgm.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_lgm.best_params_)
print("Best Precision Score:", gs_lgm.best_score_)

tuned_lgm= gs_lgm.best_estimator_
y_pred_tuned_lgm= tuned_lgm.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_lgm= tuned_lgm.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_lgm, labels= tuned_lgm.classes_)
print(classification_report(y_test, y_pred_tuned_lgm, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_lgm, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_lgm))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_lgm))

Fitting 5 folds for each of 80 candidates, totalling 400 fits
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Warning] min_data_in_leaf is set=30, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=30
[LightGBM] [Info] Number of positive: 33930, number of negative: 33930
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003586 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2550
[LightGBM] [Info] Number of data points in the train set: 67860, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
Best Parameters: {'learning_rate': 0.1, 'min_data_in_leaf': 30, 'num_leaves': 127, 'reg_alpha': 0.1}
Best Precision Score: 0.9739639806705102
[LightGBM] [Warning] min_data_in_leaf is set=30, 

--- CatBoost ---

In [35]:
from catboost import CatBoostClassifier
cat= CatBoostClassifier(random_state=15)
cat.fit(X_sm_tl_selected, y_sm_tl)
y_pred_cat= cat.predict(X_sm_tl_test_selected)
y_pred_prob_cat= cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_cat, labels= cat.classes_)
print(classification_report(y_test, y_pred_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_cat))

Learning rate set to 0.062378
0:	learn: 0.6360776	total: 172ms	remaining: 2m 51s
1:	learn: 0.5990894	total: 193ms	remaining: 1m 36s
2:	learn: 0.5692574	total: 215ms	remaining: 1m 11s
3:	learn: 0.5365817	total: 238ms	remaining: 59.2s
4:	learn: 0.5130120	total: 260ms	remaining: 51.8s
5:	learn: 0.4985634	total: 282ms	remaining: 46.7s
6:	learn: 0.4835417	total: 305ms	remaining: 43.2s
7:	learn: 0.4693816	total: 327ms	remaining: 40.5s
8:	learn: 0.4518443	total: 347ms	remaining: 38.2s
9:	learn: 0.4390304	total: 368ms	remaining: 36.4s
10:	learn: 0.4291396	total: 388ms	remaining: 34.9s
11:	learn: 0.4168746	total: 410ms	remaining: 33.7s
12:	learn: 0.4092277	total: 430ms	remaining: 32.7s
13:	learn: 0.3996810	total: 455ms	remaining: 32.1s
14:	learn: 0.3884606	total: 474ms	remaining: 31.1s
15:	learn: 0.3797659	total: 495ms	remaining: 30.4s
16:	learn: 0.3718866	total: 519ms	remaining: 30s
17:	learn: 0.3663693	total: 541ms	remaining: 29.5s
18:	learn: 0.3575382	total: 566ms	remaining: 29.2s
19:	learn:

165:	learn: 0.2253464	total: 3.5s	remaining: 17.6s
166:	learn: 0.2252049	total: 3.53s	remaining: 17.6s
167:	learn: 0.2250331	total: 3.55s	remaining: 17.6s
168:	learn: 0.2248813	total: 3.57s	remaining: 17.5s
169:	learn: 0.2247467	total: 3.59s	remaining: 17.5s
170:	learn: 0.2246028	total: 3.61s	remaining: 17.5s
171:	learn: 0.2243825	total: 3.62s	remaining: 17.4s
172:	learn: 0.2241838	total: 3.64s	remaining: 17.4s
173:	learn: 0.2240068	total: 3.66s	remaining: 17.4s
174:	learn: 0.2237614	total: 3.67s	remaining: 17.3s
175:	learn: 0.2234904	total: 3.69s	remaining: 17.3s
176:	learn: 0.2233240	total: 3.71s	remaining: 17.2s
177:	learn: 0.2232026	total: 3.72s	remaining: 17.2s
178:	learn: 0.2231593	total: 3.75s	remaining: 17.2s
179:	learn: 0.2229733	total: 3.76s	remaining: 17.1s
180:	learn: 0.2227850	total: 3.78s	remaining: 17.1s
181:	learn: 0.2225772	total: 3.8s	remaining: 17.1s
182:	learn: 0.2224512	total: 3.82s	remaining: 17.1s
183:	learn: 0.2223359	total: 3.84s	remaining: 17s
184:	learn: 0.22

326:	learn: 0.2063977	total: 6.4s	remaining: 13.2s
327:	learn: 0.2063689	total: 6.42s	remaining: 13.2s
328:	learn: 0.2063439	total: 6.44s	remaining: 13.1s
329:	learn: 0.2062491	total: 6.46s	remaining: 13.1s
330:	learn: 0.2062133	total: 6.47s	remaining: 13.1s
331:	learn: 0.2061766	total: 6.49s	remaining: 13.1s
332:	learn: 0.2061481	total: 6.51s	remaining: 13s
333:	learn: 0.2060901	total: 6.52s	remaining: 13s
334:	learn: 0.2060136	total: 6.54s	remaining: 13s
335:	learn: 0.2059680	total: 6.55s	remaining: 13s
336:	learn: 0.2058935	total: 6.57s	remaining: 12.9s
337:	learn: 0.2058229	total: 6.59s	remaining: 12.9s
338:	learn: 0.2057775	total: 6.61s	remaining: 12.9s
339:	learn: 0.2057474	total: 6.63s	remaining: 12.9s
340:	learn: 0.2057125	total: 6.64s	remaining: 12.8s
341:	learn: 0.2056460	total: 6.66s	remaining: 12.8s
342:	learn: 0.2056161	total: 6.68s	remaining: 12.8s
343:	learn: 0.2055499	total: 6.69s	remaining: 12.8s
344:	learn: 0.2055304	total: 6.71s	remaining: 12.7s
345:	learn: 0.2054361

486:	learn: 0.2001688	total: 9.24s	remaining: 9.73s
487:	learn: 0.2001440	total: 9.26s	remaining: 9.71s
488:	learn: 0.2001181	total: 9.27s	remaining: 9.69s
489:	learn: 0.2000850	total: 9.29s	remaining: 9.67s
490:	learn: 0.2000688	total: 9.3s	remaining: 9.65s
491:	learn: 0.2000535	total: 9.32s	remaining: 9.62s
492:	learn: 0.2000193	total: 9.34s	remaining: 9.6s
493:	learn: 0.2000014	total: 9.36s	remaining: 9.59s
494:	learn: 0.1999669	total: 9.38s	remaining: 9.56s
495:	learn: 0.1999520	total: 9.39s	remaining: 9.54s
496:	learn: 0.1999333	total: 9.41s	remaining: 9.52s
497:	learn: 0.1999138	total: 9.43s	remaining: 9.5s
498:	learn: 0.1998886	total: 9.45s	remaining: 9.48s
499:	learn: 0.1998544	total: 9.47s	remaining: 9.47s
500:	learn: 0.1998332	total: 9.48s	remaining: 9.45s
501:	learn: 0.1998024	total: 9.5s	remaining: 9.43s
502:	learn: 0.1997800	total: 9.52s	remaining: 9.4s
503:	learn: 0.1997613	total: 9.54s	remaining: 9.39s
504:	learn: 0.1997482	total: 9.56s	remaining: 9.37s
505:	learn: 0.199

647:	learn: 0.1967376	total: 12.1s	remaining: 6.56s
648:	learn: 0.1967326	total: 12.1s	remaining: 6.54s
649:	learn: 0.1967105	total: 12.1s	remaining: 6.52s
650:	learn: 0.1966873	total: 12.1s	remaining: 6.5s
651:	learn: 0.1966733	total: 12.1s	remaining: 6.48s
652:	learn: 0.1966460	total: 12.2s	remaining: 6.46s
653:	learn: 0.1966274	total: 12.2s	remaining: 6.44s
654:	learn: 0.1966120	total: 12.2s	remaining: 6.42s
655:	learn: 0.1965983	total: 12.2s	remaining: 6.4s
656:	learn: 0.1965892	total: 12.2s	remaining: 6.38s
657:	learn: 0.1965635	total: 12.2s	remaining: 6.36s
658:	learn: 0.1965385	total: 12.3s	remaining: 6.34s
659:	learn: 0.1965271	total: 12.3s	remaining: 6.33s
660:	learn: 0.1965184	total: 12.3s	remaining: 6.31s
661:	learn: 0.1965020	total: 12.3s	remaining: 6.29s
662:	learn: 0.1964940	total: 12.3s	remaining: 6.27s
663:	learn: 0.1964659	total: 12.4s	remaining: 6.25s
664:	learn: 0.1964550	total: 12.4s	remaining: 6.24s
665:	learn: 0.1964316	total: 12.4s	remaining: 6.22s
666:	learn: 0.

817:	learn: 0.1942203	total: 15.1s	remaining: 3.35s
818:	learn: 0.1942132	total: 15.1s	remaining: 3.33s
819:	learn: 0.1942062	total: 15.1s	remaining: 3.31s
820:	learn: 0.1942000	total: 15.1s	remaining: 3.3s
821:	learn: 0.1941825	total: 15.1s	remaining: 3.28s
822:	learn: 0.1941717	total: 15.2s	remaining: 3.26s
823:	learn: 0.1941647	total: 15.2s	remaining: 3.24s
824:	learn: 0.1941527	total: 15.2s	remaining: 3.22s
825:	learn: 0.1941392	total: 15.2s	remaining: 3.2s
826:	learn: 0.1941194	total: 15.2s	remaining: 3.18s
827:	learn: 0.1941077	total: 15.2s	remaining: 3.17s
828:	learn: 0.1940955	total: 15.3s	remaining: 3.15s
829:	learn: 0.1940736	total: 15.3s	remaining: 3.13s
830:	learn: 0.1940648	total: 15.3s	remaining: 3.11s
831:	learn: 0.1940507	total: 15.3s	remaining: 3.09s
832:	learn: 0.1940271	total: 15.3s	remaining: 3.07s
833:	learn: 0.1940200	total: 15.3s	remaining: 3.05s
834:	learn: 0.1940117	total: 15.4s	remaining: 3.03s
835:	learn: 0.1940061	total: 15.4s	remaining: 3.02s
836:	learn: 0.

980:	learn: 0.1923780	total: 17.9s	remaining: 347ms
981:	learn: 0.1923646	total: 18s	remaining: 329ms
982:	learn: 0.1923592	total: 18s	remaining: 311ms
983:	learn: 0.1923530	total: 18s	remaining: 293ms
984:	learn: 0.1923490	total: 18s	remaining: 274ms
985:	learn: 0.1923316	total: 18s	remaining: 256ms
986:	learn: 0.1923149	total: 18.1s	remaining: 238ms
987:	learn: 0.1923017	total: 18.1s	remaining: 219ms
988:	learn: 0.1922917	total: 18.1s	remaining: 201ms
989:	learn: 0.1922745	total: 18.1s	remaining: 183ms
990:	learn: 0.1922665	total: 18.1s	remaining: 165ms
991:	learn: 0.1922544	total: 18.1s	remaining: 146ms
992:	learn: 0.1922487	total: 18.2s	remaining: 128ms
993:	learn: 0.1922423	total: 18.2s	remaining: 110ms
994:	learn: 0.1922292	total: 18.2s	remaining: 91.4ms
995:	learn: 0.1922196	total: 18.2s	remaining: 73.1ms
996:	learn: 0.1922138	total: 18.2s	remaining: 54.8ms
997:	learn: 0.1922055	total: 18.2s	remaining: 36.5ms
998:	learn: 0.1922022	total: 18.2s	remaining: 18.3ms
999:	learn: 0.192

In [36]:
params_CatBoost= {
                  'depth': [3,5,10],
                  'learning_rate' : [0.01,0.1,1],
                  'iterations' : [5,10,50,100]
                 }

gs_cat= GridSearchCV(estimator= cat, param_grid= params_CatBoost, cv=5, scoring='precision', n_jobs=-1, verbose=2)
gs_cat.fit(X_sm_tl_selected, y_sm_tl)

print("Best Parameters:", gs_cat.best_params_)
print("Best Precision Score:", gs_cat.best_score_)

tuned_cat= gs_cat.best_estimator_
y_pred_tuned_cat= tuned_cat.predict(X_sm_tl_test_selected)
y_pred_prob_tuned_cat= tuned_cat.predict_proba(X_sm_tl_test_selected)[:,1]

cm= confusion_matrix(y_test, y_pred_tuned_cat, labels= tuned_cat.classes_)
print(classification_report(y_test, y_pred_tuned_cat, zero_division=0))
print('Precision:', precision_score(y_test, y_pred_tuned_cat, zero_division=0))
print('Accuracy:', accuracy_score(y_test, y_pred_tuned_cat))
print('AUC:', roc_auc_score(y_test, y_pred_prob_tuned_cat))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
0:	learn: 0.4728718	total: 9.59ms	remaining: 949ms
1:	learn: 0.4056086	total: 20.7ms	remaining: 1.01s
2:	learn: 0.3686138	total: 28.2ms	remaining: 911ms
3:	learn: 0.3449695	total: 37.4ms	remaining: 898ms
4:	learn: 0.3240337	total: 46.9ms	remaining: 891ms
5:	learn: 0.3065694	total: 56.1ms	remaining: 879ms
6:	learn: 0.2897885	total: 67.3ms	remaining: 894ms
7:	learn: 0.2802968	total: 76.7ms	remaining: 882ms
8:	learn: 0.2763340	total: 86.6ms	remaining: 876ms
9:	learn: 0.2699988	total: 97.9ms	remaining: 881ms
10:	learn: 0.2644587	total: 107ms	remaining: 865ms
11:	learn: 0.2577951	total: 118ms	remaining: 864ms
12:	learn: 0.2530890	total: 130ms	remaining: 870ms
13:	learn: 0.2480195	total: 143ms	remaining: 880ms
14:	learn: 0.2422741	total: 156ms	remaining: 882ms
15:	learn: 0.2394632	total: 166ms	remaining: 873ms
16:	learn: 0.2363969	total: 176ms	remaining: 858ms
17:	learn: 0.2334523	total: 186ms	remaining: 847ms
18:	learn: 0.2321371

--- Visualisation ---

In [37]:
precision_scores= {
                    'Logistic Regression Precision:': precision_score(y_test, y_pred_lr, zero_division= 0),
                    'Decision Tree Precision:': precision_score(y_test, y_pred_dt, zero_division= 0),
                    'Tuned Decision Tree Precision:': precision_score(y_test, y_pred_tuned_dt, zero_division= 0),
                    'KNeighborsClassifier Precision:': precision_score(y_test, y_pred_knn, zero_division= 0),
                    'Tuned KNeighborsClassifier Precision:': precision_score(y_test, y_pred_tuned_knn, zero_division= 0),
                    'GaussianNB Precision:': precision_score(y_test, y_pred_nb, zero_division= 0),
                    'SVM Precision:': precision_score(y_test, y_pred_svc, zero_division= 0),
                    'Random Forest Precision:': precision_score(y_test, y_pred_rf, zero_division= 0),
                    'Tuned Random Forest Precision:': precision_score(y_test, y_pred_tuned_rf, zero_division= 0),
                    'AdaBoost Precision:': precision_score(y_test, y_pred_ada, zero_division= 0),
                    'Tuned AdaBoost Precision:': precision_score(y_test, y_pred_tuned_ada, zero_division= 0),
                    'GradientBoosting Precision:': precision_score(y_test, y_pred_grb, zero_division= 0),
                    'Tuned GradientBoosting Precision:': precision_score(y_test, y_pred_tuned_grb, zero_division= 0),
                    'XGB Precision:': precision_score(y_test, y_pred_xgb, zero_division= 0),
                    'Tuned XGB Precision:': precision_score(y_test, y_pred_tuned_xgb, zero_division= 0),
                    'LGBM Precision:': precision_score(y_test, y_pred_lgm, zero_division= 0),
                    'Tuned LGBM Precision:': precision_score(y_test, y_pred_tuned_lgm, zero_division= 0),
                    'CatBoost Precision:': precision_score(y_test, y_pred_cat, zero_division= 0),
                    'Tuned CatBoost Precision:': precision_score(y_test, y_pred_tuned_cat, zero_division= 0)
                  }

annova_precision= pd.DataFrame(list(precision_scores.items()), columns= ['Model', 'Precision Score'])
annova_precision= annova_precision.sort_values(by= 'Precision Score', ascending=False)
print(annova_precision)

                                    Model  Precision Score
17                    CatBoost Precision:         0.455882
13                         XGB Precision:         0.454849
16                  Tuned LGBM Precision:         0.452962
15                        LGBM Precision:         0.432515
18              Tuned CatBoost Precision:         0.424699
7                Random Forest Precision:         0.397701
12      Tuned GradientBoosting Precision:         0.396382
14                   Tuned XGB Precision:         0.382353
8          Tuned Random Forest Precision:         0.381862
1                Decision Tree Precision:         0.380403
4   Tuned KNeighborsClassifier Precision:         0.378882
2          Tuned Decision Tree Precision:         0.373219
10              Tuned AdaBoost Precision:         0.359081
11            GradientBoosting Precision:         0.341365
3         KNeighborsClassifier Precision:         0.316393
9                     AdaBoost Precision:         0.3106

In [38]:
accuracy_scores= {
                    'Logistic Regression Accuracy:': accuracy_score(y_test, y_pred_lr),
                    'Decision Tree Accuracy:': accuracy_score(y_test, y_pred_dt),
                    'Tuned Decision Tree Accuracy:': accuracy_score(y_test, y_pred_tuned_dt),
                    'KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_knn),
                    'Tuned KNeighborsClassifier Accuracy:': accuracy_score(y_test, y_pred_tuned_knn),
                    'GaussianNB Accuracy:': accuracy_score(y_test, y_pred_nb),
                    'SVM Accuracy:': accuracy_score(y_test, y_pred_svc),
                    'Random Forest Accuracy:': accuracy_score(y_test, y_pred_rf),
                    'Tuned Random Forest Accuracy:': accuracy_score(y_test, y_pred_tuned_rf),
                    'AdaBoost Accuracy:': accuracy_score(y_test, y_pred_ada),
                    'Tuned AdaBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_ada),
                    'GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_grb),
                    'Tuned GradientBoosting Accuracy:': accuracy_score(y_test, y_pred_tuned_grb),
                    'XGB Accuracy:': accuracy_score(y_test, y_pred_xgb),
                    'Tuned XGB Accuracy:': accuracy_score(y_test, y_pred_tuned_xgb),
                    'LGBM Accuracy:': accuracy_score(y_test, y_pred_lgm),
                    'Tuned LGBM Accuracy:': accuracy_score(y_test, y_pred_tuned_lgm),
                    'CatBoost Accuracy:': accuracy_score(y_test, y_pred_cat),
                    'Tuned CatBoost Accuracy:': accuracy_score(y_test, y_pred_tuned_cat)
                  }

annova_accuracy= pd.DataFrame(list(accuracy_scores.items()), columns= ['Model', 'Accuracy Score'])
annova_accuracy= annova_accuracy.sort_values(by= 'Accuracy Score', ascending=False)
print(annova_accuracy)

                                   Model  Accuracy Score
17                    CatBoost Accuracy:        0.900732
16                  Tuned LGBM Accuracy:        0.900414
13                         XGB Accuracy:        0.900414
15                        LGBM Accuracy:        0.898611
18              Tuned CatBoost Accuracy:        0.897974
4   Tuned KNeighborsClassifier Accuracy:        0.895005
1                Decision Tree Accuracy:        0.894474
14                   Tuned XGB Accuracy:        0.893944
2          Tuned Decision Tree Accuracy:        0.893838
7                Random Forest Accuracy:        0.893838
8          Tuned Random Forest Accuracy:        0.892778
12      Tuned GradientBoosting Accuracy:        0.889914
3         KNeighborsClassifier Accuracy:        0.879521
10              Tuned AdaBoost Accuracy:        0.874642
11            GradientBoosting Accuracy:        0.869763
9                     AdaBoost Accuracy:        0.839432
6                          SVM 

In [39]:
auc_scores= {
                    'Logistic Regression AUC:': roc_auc_score(y_test, y_pred_prob_lr),
                    'Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_dt),
                    'Tuned Decision Tree AUC:': roc_auc_score(y_test, y_pred_prob_tuned_dt),
                    'KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_knn),
                    'Tuned KNeighborsClassifier AUC:': roc_auc_score(y_test, y_pred_prob_tuned_knn),
                    'GaussianNB AUC:': roc_auc_score(y_test, y_pred_prob_nb),
                    'SVM AUC:': roc_auc_score(y_test, y_pred_prob_svc),
                    'Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_rf),
                    'Tuned Random Forest AUC:': roc_auc_score(y_test, y_pred_prob_tuned_rf),
                    'AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_ada),
                    'Tuned AdaBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_ada),
                    'GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_grb),
                    'Tuned GradientBoosting AUC:': roc_auc_score(y_test, y_pred_prob_tuned_grb),
                    'XGB AUC:': roc_auc_score(y_test, y_pred_prob_xgb),
                    'Tuned XGB AUC:': roc_auc_score(y_test, y_pred_prob_tuned_xgb),
                    'LGBM AUC:': roc_auc_score(y_test, y_pred_prob_lgm),
                    'Tuned LGBM AUC:': roc_auc_score(y_test, y_pred_prob_tuned_lgm),
                    'CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_cat),
                    'Tuned CatBoost AUC:': roc_auc_score(y_test, y_pred_prob_tuned_cat)
                  }

annova_auc= pd.DataFrame(list(auc_scores.items()), columns= ['Model', 'AUC Score'])
annova_auc= annova_auc.sort_values(by= 'AUC Score', ascending=False)
print(annova_auc)

                              Model  AUC Score
0          Logistic Regression AUC:   0.828933
9                     AdaBoost AUC:   0.823123
10              Tuned AdaBoost AUC:   0.822377
11            GradientBoosting AUC:   0.816988
5                   GaussianNB AUC:   0.814646
12      Tuned GradientBoosting AUC:   0.810208
15                        LGBM AUC:   0.801996
17                    CatBoost AUC:   0.794154
13                         XGB AUC:   0.793390
18              Tuned CatBoost AUC:   0.792953
16                  Tuned LGBM AUC:   0.786164
6                          SVM AUC:   0.775533
14                   Tuned XGB AUC:   0.769931
8          Tuned Random Forest AUC:   0.749742
7                Random Forest AUC:   0.748707
3         KNeighborsClassifier AUC:   0.717496
1                Decision Tree AUC:   0.700878
2          Tuned Decision Tree AUC:   0.699477
4   Tuned KNeighborsClassifier AUC:   0.634219
